#VectorDB

###  ¿Qué es VectorDB?
**VectorDB** es un paquete de Python extremadamente ligero diseñado para almacenar y recuperar texto. Está optimizado para casos de uso donde la **baja latencia** es esencial.

**Características principales:**
* **Procesamiento:** Utiliza técnicas de fragmentación (chunking), incrustación (embedding) y búsqueda vectorial.
* **Gestión:** Permite guardar, buscar y gestionar datos textuales con sus metadatos asociados.
* **Interfaz:** Simple y fácil de usar.

---

###  Modelos de Embedding (Open Source)
Para convertir nuestros textos en vectores, utilizaremos estos excelentes modelos de código abierto:

1. **[all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) (Hugging Face)** * *El clásico:* Súper ligero (~100 MB), rapidísimo e ideal para empezar. Funciona con `sentence-transformers`.
2. **[bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) (Hugging Face)**
   * *El moderno:* Mismo tamaño que el anterior, pero con mayor precisión en las búsquedas.
3. **[Universal Sentence Encoder 4](https://www.kaggle.com/models/google/universal-sentence-encoder/tensorFlow2/universal-sentence-encoder/2) (Google / Kaggle)**
   * *El todoterreno:* Modelo oficial de Google basado en TensorFlow. Es muy robusto, genera vectores de 512 dimensiones y es excelente para tareas de similitud de textos generales.


In [2]:
pip install chromadb sentence-transformers pandas


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Step 1: Load your CSV file
csv_path = "/content/drive/MyDrive/Inteligencia Artificial UNAL/Sistemas Inteligentes/Practica/Semana 2/ChromaDB/movies.csv"  # Coloco la ruta donde tengo guardado los datos.
df = pd.read_csv(csv_path)

# Check for "title" field
if "title" not in df.columns:
    raise ValueError("The CSV does not contain a 'title' column.")

/tmp/ipython-input-3589769333.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


## Ejercicio

- Ahora, inserta el campo "resumen". ¿Cuánto tiempo te llevó hacerlo?
- Prueba con un par de ejemplos de búsqueda y genera un informe.
- Prueba con el mismo título de película y compara estos resultados con los de Google (codificador universal de oraciones).
- Compara el tiempo de predicción de este método con el de Google.
  


# Solución ejercicio


## **Inciso (1)**
**Enunciado:** Ahora incrusta el campo «descripción general», ¿cuánto tiempo te ha llevado hacerlo?

**Resultado:** Tardó 1 minuto y 36 segundos

In [8]:
# Step 2: Create embeddings using a SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2") # this is an embedder
texts = df["overview"].astype(str).tolist()
embeddings = model.encode(texts, show_progress_bar=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1421 [00:00<?, ?it/s]

In [9]:
# Step 3: Set up Chroma DB and collection

from tqdm import tqdm

# Initialize Chroma client
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("titles_collection")

# Batch insert (max 5000 per batch)
batch_size = 5000
for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embeddings[i:i+batch_size]
    batch_ids = [f"id_{j}" for j in range(i, i+len(batch_texts))]
    collection.add(
        documents=batch_texts,
        embeddings=batch_embeddings,
        ids=batch_ids
    )

print("All documents inserted in batches.")


100%|██████████| 10/10 [00:51<00:00,  5.11s/it]

All documents inserted in batches.


## **Inciso (2)**
**Enunciado:** Prueba un par de ejemplos de búsqueda e informa.

**Resultado:**

Usando la descripción de spiderman encontró con exito en el top 1 a que pelicula se referia.

In [17]:
import time
results = collection.query(
    query_texts=[" A teenager gets bitten by a spider and gets superpowers, but with great power comes great responsibility, and he still needs to learn that."],
    n_results=5,
    include=["documents", "distances"]  # Include relevance scores (lower is better)
)

print("Top Matches:")
for doc, score in zip(results["documents"][0], results["distances"][0]):
    probability = round(1 / (1 + score), 2)  # Convert distance to a pseudo-probability
    print(f"{doc}, confidence: {probability}")


Top Matches:
After being bitten by a genetically altered spider, nerdy high school student Peter Parker is endowed with amazing powers., confidence: 0.6
Teenagers from a small town and their high school science teacher join forces to battle a giant mutant spider, living in a cave nearby and getting hungry., confidence: 0.54
A teenager finds himself transported to an island where he must help protect a group of orphans with special powers from creatures intent on destroying them., confidence: 0.52
An 11-year-old boy becomes an unlikely superhero when he discovers that he has the ability to leave his body and fly through walls., confidence: 0.51
A boy is the only family member without superpowers in this Disney Film. The world depends on him saving his family from computerized brainwashers. Will he realize that it doesn't take superpowers to be a hero in time to help them defeat the villains?, confidence: 0.51


## **Inciso (3)**
**Enunciado:** Prueba con el mismo título de película y compara estos resultados con los de Google (codificador de frases universal).

**Resultado:**

Usando la misma descripción de spiderman demostró el mismo top 1, pero en lo demás cambió.

In [12]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("Done!")

Done!


In [14]:
df = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial UNAL/Sistemas Inteligentes/Practica/Semana 2/ChromaDB/movies.csv", nrows=45000)

df['original_title'] = df['original_title'].str.lower()

df['overview'] = (
    df['overview']
    .fillna('')
    .astype(str)
    .str.lower()
)

/tmp/ipython-input-2850954358.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial UNAL/Sistemas Inteligentes/Practica/Semana 2/ChromaDB/movies.csv", nrows=45000)


In [16]:

overviews = []

for index, row in df.iterrows():
    overviews.append(row.overview)


bank_vec = model(overviews)

mystr = 'Un adolescente es picado por una araña y obtiene superpoderes, pero un gran poder conlleva una gran responsabilidad, y él aún tiene que aprender eso.'
print(mystr)

query_vec = model([mystr])

correlation = np.inner(query_vec, bank_vec).flatten()

top_indices = np.argsort(correlation)[-5:][::-1]

print("\nTop five closest matches:")
for idx in top_indices:
    print(overviews[idx], correlation[idx])

top_matches = [(overviews[idx], correlation[idx]) for idx in top_indices]


"Un adolescente es picado por una araña y obtiene superpoderes, pero un gran poder conlleva una gran responsabilidad, y él aún tiene que aprender eso.

Top five closest matches:
los marziano reconstruye la oportunidad de un reencuentro entre dos hermanos tras un largo período de distanciamiento. se trata de un relato que trabaja con profunda inteligencia -y con rasgos de comedia- el universo de las relaciones familiares. 0.6506549
sebas es un delincuente de poca monta recién salido de prisión que decide buscar empleo en una pequeña empresa de galicia. pero no por vocación desde luego, sino porque algo le dice que el dueño de la empresa, regueira, no se ha pagado el jaguar con cangrejos. cuando por fin es contratado, regueira comprueba muy rápidamente que aquel joven de dientes mellados tiene agallas y decide convertirlo en su ayudante. pero, sebastián, tan listo y tan despiadado, ignora que forma parte de un juego que le supera... 0.6388421
¿bastan 7 minutos para enamorarse? los protag

## **Inciso (4)**
**Enunciado:** Compare the time of prediction of this approach and the one of google

**Resultado:**

Usando la vector database la predicción demoró **4 segundos**

Mientras que usando la metodología del modelo de google demoró **10 segundos**

una diferencia de **6 segundos**




#NOVEDAD

Los modelos estan mejor preentrenados con infromacion en inglés, al momento de vectorizar la infromacion y luego almacenar, luego de realizar una consulta. En inglés es mucho más precisa la respuesta, caso contrario en español, en ambos modelos tuvo muchos sesgos.


Español: Un adolescente es picado por una araña y obtiene superpoderes, pero un gran poder conlleva una gran responsabilidad, y él aún tiene que aprender eso.

* Respuesta del modelo: "Pro Dia Nascer Feliz" é o segundo longa-metragem do diretor João Jardim, diretor do cultuado documentário "Janela da Alma" que,formação - o diretor traz à tona, além de questões comuns a qualquer adolescente dentro do ambiente escolar, questões como a desigualdade social e o impacto da banalização da violência no desenvolvimento de muitos desses jovens., confidence: 0.53



Inglés: A teenager gets bitten by a spider and gets superpowers, but with great power comes great responsibility, and he still needs to learn that.

After being bitten by a genetically altered spider, nerdy high school student Peter Parker is endowed with amazing powers., confidence: 0.6

### Los modelos al ser en formato inglés la respuesta es mayor.